In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [52]:
def get_title(soup):
  try:
    product_title = soup.find('h1', attrs={'class':'-fs20 -pts -pbxs'}).text.strip()
  except AttributeError:
      product_title = ''
  return product_title

def get_price(soup):
  try:
    product_price = soup.find('span',attrs={'class':"-b -ubpt -tal -fs24 -prxs"}).text.strip()
  except AttributeError:
      product_price = ''
  return product_price

def get_stars(soup):
  try:
    product_stars = soup.find('div', attrs={'class':"stars _m _al"}).text.strip()
  except AttributeError:
      product_stars = ''
  return product_stars

def get_reviews(soup):
    try:
        review_count = soup.find("a", attrs={'class':"-plxs _more"}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count
      

In [54]:
if __name__ == '__main__':
    
    URL = "https://www.jumia.co.ke/televisions/?display_size=55.0--55"
    
    webpage = requests.get(URL)

    soup = BeautifulSoup(webpage.text, "html.parser")

    links = soup.find_all("a", attrs={'class':'core'})

    links_list = []

    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "stars":[], "reviews":[]}
    
    for link in links_list:
        new_webpage = requests.get("https://www.jumia.co.ke" + link)

        new_soup = BeautifulSoup(new_webpage.text, "html.parser")

        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['stars'].append(get_stars(new_soup))
        d['reviews'].append(get_reviews(new_soup))
 
    jumia_df = pd.DataFrame.from_dict(d)
    jumia_df['title'] = jumia_df['title'].replace('', np.nan)
    jumia_df = jumia_df.dropna(subset=['title'])
    jumia_df.to_csv("jumia TV prices 2024.csv", header=True, index=False)